In [1]:
%run 1.dataset.ipynb

Dataset salvo em ../datasets/edges.csv
     src   dest       rssi       etx      delay  busy_fraction  label
0  51679   4099 -52.240193  3.213027  29.192350       0.273040      1
1   5598  97030 -65.528683  9.078898  54.702416       0.385090      0
2  98247  27356 -57.386371  3.047537  31.362661       0.311230      1
3  59679  62407 -60.366136  1.981208  32.438587       0.193277      1
4  56906  71176 -73.647850  5.488077  41.922475       0.776727      0

Distribuição dos labels:
label
1    100000
0    100000
Name: count, dtype: int64




In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# ====== Carregar dataset ======
df = pd.read_csv("../datasets/edges.csv")

X = df[["rssi", "etx", "delay", "busy_fraction"]].values
y = df["label"].values

# Normalização
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y
)

X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test, dtype=torch.long)

# ====== Definir Modelo MLP ======
class LinkMLP(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

model = LinkMLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# ====== Treinamento ======
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(X_train)
    loss = criterion(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        pred_train = out.argmax(dim=1)
        acc_train = (pred_train == y_train).float().mean().item()
        pred_test = model(X_test).argmax(dim=1)
        acc_test = (pred_test == y_test).float().mean().item()
        print(f"Epoch {epoch}, Loss {loss.item():.4f}, "
              f"Train Acc {acc_train:.4f}, Test Acc {acc_test:.4f}")

# ====== Avaliação final ======
from sklearn.metrics import classification_report, confusion_matrix

model.eval()
with torch.no_grad():
    logits = model(X_test)
    pred_test = logits.argmax(dim=1)

# Acurácia
acc_test = (pred_test == y_test).float().mean().item()
print(f"\nAcurácia final no conjunto de teste: {acc_test:.4f}")

# Métricas adicionais
y_true = y_test.numpy()
y_pred = pred_test.numpy()

print("\nMatriz de Confusão:")
print(confusion_matrix(y_true, y_pred))

print("\nRelatório de Classificação:")
print(classification_report(y_true, y_pred, target_names=["Ruim", "Bom"]))

# ====== Salvar modelo ======
os.makedirs("../models", exist_ok=True)
torch.save(model.state_dict(), "../models/mlp.pth")
joblib.dump(scaler, "../models/scaler.pkl")

print("Modelo salvo em ../models/mlp.pth")
print("Modelo salvo em ../models/scaler.pkl")
print("\n")

Epoch 0, Loss 0.6177, Train Acc 0.5618, Test Acc 0.9068
Epoch 10, Loss 0.2960, Train Acc 0.9688, Test Acc 0.9691
Epoch 20, Loss 0.1142, Train Acc 0.9766, Test Acc 0.9768
Epoch 30, Loss 0.0608, Train Acc 0.9809, Test Acc 0.9815
Epoch 40, Loss 0.0452, Train Acc 0.9840, Test Acc 0.9843

Acurácia final no conjunto de teste: 0.9858

Matriz de Confusão:
[[29315   685]
 [  166 29834]]

Relatório de Classificação:
              precision    recall  f1-score   support

        Ruim       0.99      0.98      0.99     30000
         Bom       0.98      0.99      0.99     30000

    accuracy                           0.99     60000
   macro avg       0.99      0.99      0.99     60000
weighted avg       0.99      0.99      0.99     60000

Modelo salvo em ../models/mlp.pth
Modelo salvo em ../models/scaler.pkl


